In [1]:
import retrieval.cache as cache
import signals.ema as ema
import signals.simple_scalping as simple_scalping

import numpy as np
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from datetime import datetime
from tqdm import tqdm

### Make sure we have some data to work with

In [2]:
ticker = 'BTC-USD'
start = '2023-01-01'
end = '2023-12-31'
interval = '1d'

if not cache.exist_sql_db(ticker, interval):
    print(f'''Missing {ticker} {interval} in SQL DB... adding now.''')
    cache.cache_ticker(ticker, interval, start, end)

df = cache.load_ticker(ticker, interval, start, end)

#df

### Add TA Columns

In [3]:
df['EMA Slow'] = ta.ema(df['Close'], length=50)
df['EMA Fast'] = ta.ema(df['Close'], length=30)
df['RSI'] = ta.rsi(df['Close'], length=10)

# Average True Range (or Volatility) - help to define stop loss / take profit distance
df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=7)

# Bollinger bands
df=df.join(ta.bbands(df['Close'], length=15, std=1.5))

#df


### EMA signal function

In [4]:
tqdm.pandas()

df.reset_index(inplace=True)

# row.name is the index of the row -> 'Date' column
df['EMA Signal'] = df.progress_apply(
  lambda row: ema.two_above_or_below(df, row.name, 7) if row.name >= 20 else 0, 
  axis=1
)

#df

100%|██████████| 364/364 [00:00<00:00, 3841.08it/s]


### Sell By Signal

### Calculate the Simple Scalping Signal

In [ ]:
df['Simple Scalp'] = df.progress_apply(lambda row: simple_scalping.simple_signal(df, row.name, 7), axis=1)

#df[df['Simple Scalp'] != 0].head(20)

### Create Dots for Simple Scalping Signals

In [ ]:
def render_short_long_signal_point(row: pd.Series, column_name:str) -> str:
  if row[column_name] == simple_scalping.LONG_SIGNAL:
    return row['Low']-1e-3
  elif row[column_name] == simple_scalping.SHORT_SIGNAL:
    return row['High']+1e-3
  
  return np.nan

df['ss_point'] = df.apply(lambda row: render_short_long_signal_point(row, 'Simple Scalp'), axis=1)

### Plot the Graph

In [ ]:
partition = df

fig = go.Figure(data=[go.Candlestick(x=partition.index,
                open=partition['Open'],
                high=partition['High'],
                low=partition['Low'],
                close=partition['Close']),
                
                go.Scatter(x=partition.index, y=partition['BBL_15_1.5'], 
                           line=dict(color='blue', width=1), 
                           name='BBL'),

                go.Scatter(x=partition.index, y=partition['BBU_15_1.5'],
                            line=dict(color='blue', width=1),
                            name='BBU'),

                go.Scatter(x=partition.index, y=partition['EMA Slow'],
                            line=dict(color='orange', width=1),
                            name='EMA Slow'),

                go.Scatter(x=partition.index, y=partition['EMA Fast'],
                            line=dict(color='green', width=1),
                            name='EMA Fast'),
                ])

fig.add_trace(go.Scatter(x=partition.index, y=partition['ss_point'],
                            mode='markers', marker=dict(color='darkorange', size=5),
                            name='Simple Scalp'))